In [3]:
pip install scikit-learn

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 1.5 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/11.0 MB 1.3 MB/s eta 0:00:08
   --- ------------------------------------ 1.0/11.0 MB 1.4 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.0 MB 1.2 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.0 MB 1.2 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.0 MB 1.2 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.0 MB 1.2 MB/s eta 0:00:08
   ----- -------------------------

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import string
import os

## Lets load the dataset

In [17]:
df = pd.read_csv(os.getcwd()+'\\incidents_reseau.csv')
df

,ID d'incident,Timestamp,Type d'incident,Description,Severité,Statut,Responsable
0,1,2023-09-24,Intrusion,échec mise jour logiciel,Élevée,Ouvert,Equipe Réseau
1,2,2023-07-12,Panne matérielle,mise jour firmware a échoué,Élevée,En cours,Equipe Sécurité
2,3,2023-05-26,Intrusion,carte réseau ne répond plus,Faible,En cours,Equipe Réseau
3,4,2023-04-16,Intrusion,échec mise jour logiciel,Moyenne,Ouvert,Equipe Support
4,5,2023-10-22,Intrusion,échec mise jour logiciel,Faible,Ouvert,Equipe Réseau
...,...,...,...,...,...,...,...
2995,2996,2023-10-09,Mise à jour échouée,connexion au vpn a échoué,Faible,Ouvert,Equipe Support
2996,2997,2023-11-14,Mise à jour échouée,malware a été trouvé sur serveur,Faible,Résolu,Equipe Support
2997,2998,2023-12-13,Problème de connexion,capacité bande passante dépassée,Critique,Résolu,Equipe Sécurité
2998,2999,2023-03-20,Mise à jour échouée,échec mise jour logiciel,Critique,Résolu,Equipe Support


## Data Preprocessing

In [18]:
# Convertir les timestamps en datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [19]:
# Traitement de texte
# Télécharger les stopwords de NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))
stemmer = SnowballStemmer("french")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cx3u\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Supprimer les stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Appliquer le stemming
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

df['Description'] = df['Description'].apply(preprocess_text)

In [21]:
df

,ID d'incident,Timestamp,Type d'incident,Description,Severité,Statut,Responsable
0,1,2023-09-24,Intrusion,échec mis jour logiciel,Élevée,Ouvert,Equipe Réseau
1,2,2023-07-12,Panne matérielle,mis jour firmwar a échou,Élevée,En cours,Equipe Sécurité
2,3,2023-05-26,Intrusion,cart réseau répond plus,Faible,En cours,Equipe Réseau
3,4,2023-04-16,Intrusion,échec mis jour logiciel,Moyenne,Ouvert,Equipe Support
4,5,2023-10-22,Intrusion,échec mis jour logiciel,Faible,Ouvert,Equipe Réseau
...,...,...,...,...,...,...,...
2995,2996,2023-10-09,Mise à jour échouée,connexion vpn a échou,Faible,Ouvert,Equipe Support
2996,2997,2023-11-14,Mise à jour échouée,malwar a trouv serveur,Faible,Résolu,Equipe Support
2997,2998,2023-12-13,Problème de connexion,capac band pass dépass,Critique,Résolu,Equipe Sécurité
2998,2999,2023-03-20,Mise à jour échouée,échec mis jour logiciel,Critique,Résolu,Equipe Support


In [22]:
# Vectorisation des descriptions
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Description'])

## EDA


## Train Test Split


In [23]:
# Target
y = df["Type d'incident"]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


## Model Training

In [25]:
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

In [26]:

models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}# Entrainer et evaluer chaque modele
results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results[model_name] = {
        "accuracy": accuracy,
        "report": report
    }

In [27]:
# Results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print(f"Accuracy: {metrics['accuracy']}")
    print("Classification Report:")
    print(metrics['report'])
    print("="*60)

Model: Naive Bayes
Accuracy: 0.16777777777777778
Classification Report:
                       precision    recall  f1-score   support

            Intrusion       0.20      0.28      0.24       175
  Mise à jour échouée       0.12      0.17      0.14       180
     Panne matérielle       0.22      0.13      0.16       183
Problème de connexion       0.17      0.10      0.13       177
            Surcharge       0.16      0.16      0.16       185

             accuracy                           0.17       900
            macro avg       0.17      0.17      0.17       900
         weighted avg       0.17      0.17      0.16       900

Model: Logistic Regression
Accuracy: 0.16444444444444445
Classification Report:
                       precision    recall  f1-score   support

            Intrusion       0.18      0.27      0.22       175
  Mise à jour échouée       0.12      0.17      0.14       180
     Panne matérielle       0.22      0.07      0.11       183
Problème de connexion    

## Pickling The Model file For Deployment

In [28]:
import pickle

In [29]:
for model_name, model in models.items():
    path = os.path.join(os.getcwd(), f"{model_name}_model.pkl")
    with open(path , 'wb') as file:
        pickle.dump(model, file)

# Sauvegarder le vectorizer
path_vectorizer = os.path.join(os.getcwd(), f"vectorizer.pkl")
with open(path_vectorizer, 'wb') as file:
    pickle.dump(vectorizer, file)

print("Models and vectorizer have been saved.")

Models and vectorizer have been saved.
